# Memory

## Setup

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-5")

In [ ]:
from dataclasses import dataclass


# Setup your agent runtime context
@dataclass
class Context:
    first: str
    last: str

In [ ]:
import re
from dataclasses import asdict

from langchain_core.tools import tool
from langgraph.runtime import get_runtime


@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results.

    Named parameters like :first and :last will be filled from runtime context.
    """
    runtime = get_runtime(Context)
    try:
        placeholders = re.findall(r":(\w+)", query)

        if placeholders:
            return db.run(query, parameters=asdict(runtime.context or {}))
        else:
            return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [ ]:
import requests
from langchain_core.tools import tool

FALLBACK_RATES = {"USD": 1.0, "EUR": 0.92, "JPY": 150.0, "GBP": 0.79}


@tool
def convert_currency(amount: float, to_currency: str) -> float:
    """Convert an amount in USD to another currency using live exchange rates.

    Always use this tool if the user requests a currency different
    from the one stored in the database (USD).
    """
    try:
        url = f"https://api.exchangerate.host/convert?from=USD&to={to_currency}&amount={amount}"
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()
        result = data.get("result")
        if result is not None:
            return round(float(result), 2)
    except Exception as e:
        print(f"[convert_currency] Falling back due to error: {e}")

    if to_currency not in FALLBACK_RATES:
        raise ValueError("Unsupported currency in fallback mode")
    return round(amount * FALLBACK_RATES[to_currency], 2)

In [ ]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If a query involves a specific customer, always use the named parameters
  :first and :last (instead of hard-coding values).
  Example: SELECT InvoiceId, Total FROM Invoice
           JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
           WHERE Customer.FirstName = :first AND Customer.LastName = :last;
  Assume the customer first and last name are known and will be inserted into the query later.
"""

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model=model,
    tools=[execute_sql, convert_currency],
    system_prompt=SYSTEM_PROMPT,
    context_schema=Context,
)

## Repeated Queries

In [ ]:
question = "What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=Context(first="Frank", last="Harris"),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_f2V7A6W8Yr3CdAa8mgQtNzFn)
 Call ID: call_f2V7A6W8Yr3CdAa8mgQtNzFn
  Args:
    query: SELECT Invoice.InvoiceId, Invoice.InvoiceDate, Invoice.Total
FROM Invoice
JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
WHERE Customer.FirstName = :first AND Customer.LastName = :last
ORDER BY Invoice.InvoiceDate DESC, Invoice.InvoiceId DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[(374, '2013-07-04 00:00:00', 5.94)]
================================== Ai Message ==================================

Your last invoice total was $5.94 (USD).


In [ ]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=Context(first="Frank", last="Harris"),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

Could you clarify which titles you mean?
- Album titles, track names, playlist names, or employee job titles?
- Any specific artist, customer, or time period to filter by?
- How many results would you like? (Default is 5)


## Add memory

In [10]:
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model=model,
    tools=[execute_sql, convert_currency],
    system_prompt=SYSTEM_PROMPT,
    context_schema=Context,
    checkpointer=InMemorySaver(),
)

In [ ]:
question = "What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=Context(first="Frank", last="Harris"),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_sUHf5taP2x7PiexP3Am7Rnmy)
 Call ID: call_sUHf5taP2x7PiexP3Am7Rnmy
  Args:
    query: SELECT Invoice.Total AS Total
FROM Invoice
JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
WHERE Customer.FirstName = :first AND Customer.LastName = :last
ORDER BY Invoice.InvoiceDate DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[(5.94,)]
================================== Ai Message ==================================

Your most recent invoice total was $5.94 USD.


In [ ]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=Context(first="Frank", last="Harris"),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_2izsoTAeEYSTDVeI0wZSGrIz)
 Call ID: call_2izsoTAeEYSTDVeI0wZSGrIz
  Args:
    query: SELECT Track.Name AS Title
FROM InvoiceLine
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
WHERE InvoiceLine.InvoiceId = (
  SELECT Invoice.InvoiceId
  FROM Invoice
  JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
  WHERE Customer.FirstName = :first AND Customer.LastName = :last
  ORDER BY Invoice.InvoiceDate DESC
  LIMIT 1
)
ORDER BY InvoiceLine.InvoiceLineId
LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou',), ('Through The Never',), ('My Friend Of Misery',), ('The Wait',), ('Blitzkrieg',)]
================================== Ai Message ==================================

Here are the first 5 tra